In [9]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function 
'''
using for python 3.x keywords and features in python 2. > for compatibility 
'''
from torch import nn, optim, cuda 
from torch.utils import data 
'''
in this module, there are Dataset and DataLoader ,, etc.
data.Dataset >> is using for virtual class for custom data
data.DataLoader >> is using for input Pipeline
'''
from torchvision import datasets, transforms 
'''
The torchvision package consists of popular datasets, model architectures, 
and common image transformations for computer vision.
'''
import torch.nn.functional as F 
import time

#Training settings 
batch_size = 64 
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST on {device}\n {"="*40}')

#MNIST Dataset 
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='./mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())

#Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                            batch_size = batch_size,
                            shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             shuffle=False)

class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = nn.Linear(784,520)
        self.l2 = nn.Linear(520,320)
        self.l3 = nn.Linear(320,240)
        self.l4 = nn.Linear(240,120)
        self.l5 = nn.Linear(120,10)
        
    def forward(self,x):
        x = x.view(-1,784) # Flatten the data (n,1,28,28) -> (n,784) 
        ## i think > n is batch_size,, (?)
        ## such as view of DataBase (?) 
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
    
model = Net()
model.to(device)  
## i think > it send model to device
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5)


def train(epoch):
    model.train()  # >> i think ,, overriding
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #sending data and target to device
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0: 
            print('Train Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item())) 
      
    '''
        Reference) 
        
        for i,data in enumerate(train_loader,0):
            #get the inputs 
            inputs, labels = data
        
        the data shape looks like,, 
        [tensor([[],[],...,[]]),tensor([[],[],[],...,[]])]
        for inputs and then for labels 
        
        >>>  list(tensor(inputs), tensor(data)) <<< 
        type(data) >> class 'list'
        type(data[0]) >> Torch.Tensor
    '''
            
def test():
    model.eval()
    test_loss = 0 
    correct = 0
    for data, target in test_loader : 
        data, target = data.to(device), target.to(device)
        output = model(data)
        #sum up batch loss 
        test_loss += criterion(output,target).item()
        #get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        '''
        ###### ============== ########
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        ''' 
    test_loss /= len(test_loader.dataset)
    print(f'======================\nTest set: Average loss : {test_loss:.4f},Accuracy:{correct}/{len(test_loader.dataset)}'
         f'({100.*correct / len(test_loader.dataset):.0f}%)')
    
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1,10):
        epoch_start = time.time()
        train(epoch)
        m,s = divmod(time.time() - epoch_start, 60)
        print(f'Training time : {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')
      
    '''
       takes two parameters x and y, where x is treated as 
       numerator and y is treated as denominator. 
       The method calculates both x / y and x % y and 
       returns both the values
       
       divmod(x, y)
       >>>
       (x / y, x % y)
       
    '''
        
    m,s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s \n Model was trained on {device}!')
        
      


Training MNIST on cpu
Train Epoch : 1 | Batch Status : 0/60000 (0%) | Loss : 2.303868
Train Epoch : 1 | Batch Status : 640/60000 (1%) | Loss : 2.305334
Train Epoch : 1 | Batch Status : 1280/60000 (2%) | Loss : 2.310933
Train Epoch : 1 | Batch Status : 1920/60000 (3%) | Loss : 2.298598
Train Epoch : 1 | Batch Status : 2560/60000 (4%) | Loss : 2.298925
Train Epoch : 1 | Batch Status : 3200/60000 (5%) | Loss : 2.307734
Train Epoch : 1 | Batch Status : 3840/60000 (6%) | Loss : 2.299220
Train Epoch : 1 | Batch Status : 4480/60000 (7%) | Loss : 2.305486
Train Epoch : 1 | Batch Status : 5120/60000 (9%) | Loss : 2.301051
Train Epoch : 1 | Batch Status : 5760/60000 (10%) | Loss : 2.308271
Train Epoch : 1 | Batch Status : 6400/60000 (11%) | Loss : 2.310976
Train Epoch : 1 | Batch Status : 7040/60000 (12%) | Loss : 2.294538
Train Epoch : 1 | Batch Status : 7680/60000 (13%) | Loss : 2.293129
Train Epoch : 1 | Batch Status : 8320/60000 (14%) | Loss : 2.306608
Train Epoch : 1 | Batch Status : 8960/6

Train Epoch : 2 | Batch Status : 14720/60000 (25%) | Loss : 0.737786
Train Epoch : 2 | Batch Status : 15360/60000 (26%) | Loss : 0.678977
Train Epoch : 2 | Batch Status : 16000/60000 (27%) | Loss : 0.795070
Train Epoch : 2 | Batch Status : 16640/60000 (28%) | Loss : 0.641177
Train Epoch : 2 | Batch Status : 17280/60000 (29%) | Loss : 0.613691
Train Epoch : 2 | Batch Status : 17920/60000 (30%) | Loss : 0.993400
Train Epoch : 2 | Batch Status : 18560/60000 (31%) | Loss : 0.506202
Train Epoch : 2 | Batch Status : 19200/60000 (32%) | Loss : 0.567699
Train Epoch : 2 | Batch Status : 19840/60000 (33%) | Loss : 0.613622
Train Epoch : 2 | Batch Status : 20480/60000 (34%) | Loss : 0.776030
Train Epoch : 2 | Batch Status : 21120/60000 (35%) | Loss : 0.613235
Train Epoch : 2 | Batch Status : 21760/60000 (36%) | Loss : 0.549033
Train Epoch : 2 | Batch Status : 22400/60000 (37%) | Loss : 0.808799
Train Epoch : 2 | Batch Status : 23040/60000 (38%) | Loss : 0.686920
Train Epoch : 2 | Batch Status : 2

Train Epoch : 3 | Batch Status : 30080/60000 (50%) | Loss : 0.254499
Train Epoch : 3 | Batch Status : 30720/60000 (51%) | Loss : 0.334025
Train Epoch : 3 | Batch Status : 31360/60000 (52%) | Loss : 0.616335
Train Epoch : 3 | Batch Status : 32000/60000 (53%) | Loss : 0.310601
Train Epoch : 3 | Batch Status : 32640/60000 (54%) | Loss : 0.477733
Train Epoch : 3 | Batch Status : 33280/60000 (55%) | Loss : 0.368253
Train Epoch : 3 | Batch Status : 33920/60000 (57%) | Loss : 0.156369
Train Epoch : 3 | Batch Status : 34560/60000 (58%) | Loss : 0.409191
Train Epoch : 3 | Batch Status : 35200/60000 (59%) | Loss : 0.319173
Train Epoch : 3 | Batch Status : 35840/60000 (60%) | Loss : 0.116770
Train Epoch : 3 | Batch Status : 36480/60000 (61%) | Loss : 0.242796
Train Epoch : 3 | Batch Status : 37120/60000 (62%) | Loss : 0.202575
Train Epoch : 3 | Batch Status : 37760/60000 (63%) | Loss : 0.317173
Train Epoch : 3 | Batch Status : 38400/60000 (64%) | Loss : 0.230161
Train Epoch : 3 | Batch Status : 3

Train Epoch : 4 | Batch Status : 45440/60000 (76%) | Loss : 0.096169
Train Epoch : 4 | Batch Status : 46080/60000 (77%) | Loss : 0.294423
Train Epoch : 4 | Batch Status : 46720/60000 (78%) | Loss : 0.122529
Train Epoch : 4 | Batch Status : 47360/60000 (79%) | Loss : 0.465161
Train Epoch : 4 | Batch Status : 48000/60000 (80%) | Loss : 0.096599
Train Epoch : 4 | Batch Status : 48640/60000 (81%) | Loss : 0.184897
Train Epoch : 4 | Batch Status : 49280/60000 (82%) | Loss : 0.136264
Train Epoch : 4 | Batch Status : 49920/60000 (83%) | Loss : 0.225659
Train Epoch : 4 | Batch Status : 50560/60000 (84%) | Loss : 0.120586
Train Epoch : 4 | Batch Status : 51200/60000 (85%) | Loss : 0.304009
Train Epoch : 4 | Batch Status : 51840/60000 (86%) | Loss : 0.336451
Train Epoch : 4 | Batch Status : 52480/60000 (87%) | Loss : 0.247274
Train Epoch : 4 | Batch Status : 53120/60000 (88%) | Loss : 0.271828
Train Epoch : 4 | Batch Status : 53760/60000 (90%) | Loss : 0.091608
Train Epoch : 4 | Batch Status : 5

Train Epoch : 6 | Batch Status : 640/60000 (1%) | Loss : 0.168997
Train Epoch : 6 | Batch Status : 1280/60000 (2%) | Loss : 0.212894
Train Epoch : 6 | Batch Status : 1920/60000 (3%) | Loss : 0.098921
Train Epoch : 6 | Batch Status : 2560/60000 (4%) | Loss : 0.099571
Train Epoch : 6 | Batch Status : 3200/60000 (5%) | Loss : 0.137932
Train Epoch : 6 | Batch Status : 3840/60000 (6%) | Loss : 0.156019
Train Epoch : 6 | Batch Status : 4480/60000 (7%) | Loss : 0.075641
Train Epoch : 6 | Batch Status : 5120/60000 (9%) | Loss : 0.132191
Train Epoch : 6 | Batch Status : 5760/60000 (10%) | Loss : 0.201427
Train Epoch : 6 | Batch Status : 6400/60000 (11%) | Loss : 0.114991
Train Epoch : 6 | Batch Status : 7040/60000 (12%) | Loss : 0.042541
Train Epoch : 6 | Batch Status : 7680/60000 (13%) | Loss : 0.076706
Train Epoch : 6 | Batch Status : 8320/60000 (14%) | Loss : 0.114477
Train Epoch : 6 | Batch Status : 8960/60000 (15%) | Loss : 0.125429
Train Epoch : 6 | Batch Status : 9600/60000 (16%) | Loss 

Train Epoch : 7 | Batch Status : 16000/60000 (27%) | Loss : 0.099695
Train Epoch : 7 | Batch Status : 16640/60000 (28%) | Loss : 0.117777
Train Epoch : 7 | Batch Status : 17280/60000 (29%) | Loss : 0.205355
Train Epoch : 7 | Batch Status : 17920/60000 (30%) | Loss : 0.149270
Train Epoch : 7 | Batch Status : 18560/60000 (31%) | Loss : 0.088745
Train Epoch : 7 | Batch Status : 19200/60000 (32%) | Loss : 0.180287
Train Epoch : 7 | Batch Status : 19840/60000 (33%) | Loss : 0.048204
Train Epoch : 7 | Batch Status : 20480/60000 (34%) | Loss : 0.103294
Train Epoch : 7 | Batch Status : 21120/60000 (35%) | Loss : 0.148399
Train Epoch : 7 | Batch Status : 21760/60000 (36%) | Loss : 0.099535
Train Epoch : 7 | Batch Status : 22400/60000 (37%) | Loss : 0.061254
Train Epoch : 7 | Batch Status : 23040/60000 (38%) | Loss : 0.135413
Train Epoch : 7 | Batch Status : 23680/60000 (39%) | Loss : 0.205200
Train Epoch : 7 | Batch Status : 24320/60000 (41%) | Loss : 0.030849
Train Epoch : 7 | Batch Status : 2

Train Epoch : 8 | Batch Status : 31360/60000 (52%) | Loss : 0.116606
Train Epoch : 8 | Batch Status : 32000/60000 (53%) | Loss : 0.158038
Train Epoch : 8 | Batch Status : 32640/60000 (54%) | Loss : 0.100618
Train Epoch : 8 | Batch Status : 33280/60000 (55%) | Loss : 0.135251
Train Epoch : 8 | Batch Status : 33920/60000 (57%) | Loss : 0.045641
Train Epoch : 8 | Batch Status : 34560/60000 (58%) | Loss : 0.084241
Train Epoch : 8 | Batch Status : 35200/60000 (59%) | Loss : 0.080261
Train Epoch : 8 | Batch Status : 35840/60000 (60%) | Loss : 0.054756
Train Epoch : 8 | Batch Status : 36480/60000 (61%) | Loss : 0.156278
Train Epoch : 8 | Batch Status : 37120/60000 (62%) | Loss : 0.204552
Train Epoch : 8 | Batch Status : 37760/60000 (63%) | Loss : 0.088695
Train Epoch : 8 | Batch Status : 38400/60000 (64%) | Loss : 0.137140
Train Epoch : 8 | Batch Status : 39040/60000 (65%) | Loss : 0.081570
Train Epoch : 8 | Batch Status : 39680/60000 (66%) | Loss : 0.257832
Train Epoch : 8 | Batch Status : 4

Train Epoch : 9 | Batch Status : 46720/60000 (78%) | Loss : 0.157532
Train Epoch : 9 | Batch Status : 47360/60000 (79%) | Loss : 0.074573
Train Epoch : 9 | Batch Status : 48000/60000 (80%) | Loss : 0.132287
Train Epoch : 9 | Batch Status : 48640/60000 (81%) | Loss : 0.101689
Train Epoch : 9 | Batch Status : 49280/60000 (82%) | Loss : 0.121406
Train Epoch : 9 | Batch Status : 49920/60000 (83%) | Loss : 0.069355
Train Epoch : 9 | Batch Status : 50560/60000 (84%) | Loss : 0.075064
Train Epoch : 9 | Batch Status : 51200/60000 (85%) | Loss : 0.113196
Train Epoch : 9 | Batch Status : 51840/60000 (86%) | Loss : 0.122460
Train Epoch : 9 | Batch Status : 52480/60000 (87%) | Loss : 0.074134
Train Epoch : 9 | Batch Status : 53120/60000 (88%) | Loss : 0.013727
Train Epoch : 9 | Batch Status : 53760/60000 (90%) | Loss : 0.071198
Train Epoch : 9 | Batch Status : 54400/60000 (91%) | Loss : 0.170724
Train Epoch : 9 | Batch Status : 55040/60000 (92%) | Loss : 0.050941
Train Epoch : 9 | Batch Status : 5

In [6]:
import torch
x = torch.randn(4,4)
y = x.view(16)
z = x.view(-1,8)

print(x,'\n',y,'\n',z)

tensor([[ 0.1663,  1.1857,  1.0400,  0.5672],
        [-1.5602, -0.8451,  0.3930, -0.1837],
        [ 0.2274, -0.4366, -1.2270, -0.2365],
        [-0.1978,  0.4887,  1.2223,  0.7826]]) 
 tensor([ 0.1663,  1.1857,  1.0400,  0.5672, -1.5602, -0.8451,  0.3930, -0.1837,
         0.2274, -0.4366, -1.2270, -0.2365, -0.1978,  0.4887,  1.2223,  0.7826]) 
 tensor([[ 0.1663,  1.1857,  1.0400,  0.5672, -1.5602, -0.8451,  0.3930, -0.1837],
        [ 0.2274, -0.4366, -1.2270, -0.2365, -0.1978,  0.4887,  1.2223,  0.7826]])
